This project demonstrates how to use sql magic to write SQL queries in a Jupyter notebook. Because some DBMS cannot be used to present data insights or analysis, you may want to use SQL magic on your notebook to create your presentation.

Project Outcome:
*  Learn how to connect to an existing SQLite Database
*  How to use SQL magic in Jupyter Notebook
*  Write queries to answer some questions

Required Packages
1. SQLite3 [ pip install sqlite ] (This should be available already, import sqlite3 to confirm)
2. SQLAlchemy [ pip install SQLAlchemy ]
3. Pandas [ pip install pandas ]
4. SQL Kernel [ pip install ipython-sql] 

# Personal Medical Insurance Analysis

### Attribute Information
age: age of primary beneficiary

sex: insurance contractor gender, female, male

bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height,
objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9

children: Number of children covered by health insurance / Number of dependents

smoker: Smoking

region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.

charges: Individual medical costs billed by health insurance

In [7]:
import sqlite3

In [8]:
!pip install SQLAlchemy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import pandas as pd

In [10]:
%reload_ext sql
%sql sqlite:///insurance.db


'Connected: @insurance.db'

In [11]:
# Data Preview
%sql SELECT * FROM insurance LIMIT 5

 * sqlite:///insurance.db
Done.


age,sex,bmi,children,smoker,region,charges
19,female,27.9,0,yes,southwest,16884.924
18,male,33.77,1,no,southeast,1725.5523
28,male,33,3,no,southeast,4449.462
33,male,22.705,0,no,northwest,21984.47061
32,male,28.88,0,no,northwest,3866.8552


1. How many females are smokers?

In [12]:
%%sql 
SELECT  COUNT(*)count
FROM insurance
WHERE sex = 'female' AND smoker = 'yes' 

 * sqlite:///insurance.db
Done.


count
115


2. How many males are smokers?






In [13]:
%%sql 
SELECT  COUNT(*)count
FROM insurance
WHERE sex = 'male' AND smoker = 'yes'

 * sqlite:///insurance.db
Done.


count
159


3. Show the distribution of smokers and non-smokers for each region.

In [14]:
%%sql
SELECT region, smoker, COUNT(*) as count
FROM insurance
GROUP BY region, smoker

 * sqlite:///insurance.db
Done.


region,smoker,count
northeast,no,257
northeast,yes,67
northwest,no,267
northwest,yes,58
southeast,no,273
southeast,yes,91
southwest,no,267
southwest,yes,58


4. Is there a region where its average charge is below the overall average charge?

In [15]:
#Average charge for each region
%%sql
SELECT region, AVG(charges)avg_charge
FROM insurance
GROUP BY region

 * sqlite:///insurance.db
Done.


region,avg_charge
northeast,13406.3845163858
northwest,12417.575373969228
southeast,14735.411437609895
southwest,12346.93737729231


In [16]:
#Average charge for the whole dataset
%%sql
SELECT AVG(charges)overall_avg_charge
FROM insurance

 * sqlite:///insurance.db
Done.


overall_avg_charge
13270.422265141257


In [17]:
#With CTE
%%sql
with overall AS(
  SELECT AVG(charges) overall_avg_charge
  FROM insurance
)

SELECT region, AVG(charges)avg_charge
FROM insurance, overall
GROUP BY region 
HAVING AVG(charges) < overall_avg_charge

 * sqlite:///insurance.db
Done.


region,avg_charge
northwest,12417.575373969228
southwest,12346.93737729231


In [18]:
#With subquery
%%sql
SELECT region,AVG(charges) avg_charge
FROM insurance
GROUP BY region
HAVING AVG (charges) < (SELECT AVG(charge) FROM insurance)

 * sqlite:///insurance.db
(sqlite3.OperationalError) no such column: charge
[SQL: SELECT region,AVG(charges) avg_charge
FROM insurance
GROUP BY region
HAVING AVG (charges) < (SELECT AVG(charge) FROM insurance)]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [19]:
#Returns bool value
%%sql
SELECT  region, AVG(charges) < (SELECT AVG(charges) FROM insurance) avg_less_than_overall
FROM insurance
GROUP BY  region

 * sqlite:///insurance.db
Done.


region,avg_less_than_overall
northeast,0
northwest,1
southeast,0
southwest,1


5a. What can you say about people with children and those without children in terms of smoking

In [20]:

%%sql
WITH children AS (
    SELECT  *
    FROM    insurance
    WHERE   children > 0
)

SELECT      region, sex, smoker, COUNT(*) count
FROM        children
GROUP BY    region, sex, smoker
ORDER BY    count DESC
LIMIT       10

 * sqlite:///insurance.db
Done.


region,sex,smoker,count
southeast,female,no,82
southwest,female,no,82
northwest,female,no,80
northwest,male,no,79
southeast,male,no,74
southwest,male,no,72
northeast,male,no,69
northeast,female,no,67
southeast,male,yes,34
northeast,female,yes,22


5b. What can you say about people with children and those without children in terms of smoking

In [21]:
%%sql
WITH no_children AS (
    SELECT  * 
    FROM    insurance
    WHERE   children <= 0
)

SELECT      region, sex, smoker, COUNT(*) count
FROM        no_children
GROUP BY    region, sex, smoker
ORDER BY    count DESC
LIMIT       10

 * sqlite:///insurance.db
Done.


region,sex,smoker,count
northeast,female,no,65
southeast,male,no,60
southwest,female,no,59
southeast,female,no,57
northeast,male,no,56
northwest,female,no,55
southwest,male,no,54
northwest,male,no,53
southeast,male,yes,21
northeast,male,yes,19


--6. Which region has the highest number of smokers?

In [22]:

%%sql
SELECT region, MAX(count) max_smoker
FROM    (
    SELECT      region, COUNT(*) count
    FROM        insurance
    WHERE       smoker = 'yes'
    GROUP BY    region
)

 * sqlite:///insurance.db
Done.


region,max_smoker
southeast,91


7. Which region has the lowest number of smokers?

In [23]:
%%sql
SELECT      region, COUNT(*) count
FROM        insurance
WHERE       smoker = 'yes'
GROUP BY    region

%%sql
SELECT region, MIN(count) min_smoker
FROM    (
    SELECT      region, COUNT(*) count
    FROM        insurance
    WHERE       smoker = 'yes'
    GROUP BY    region
)

 * sqlite:///insurance.db
(sqlite3.OperationalError) near "%": syntax error
[SQL: SELECT      region, COUNT(*) count
FROM        insurance
WHERE       smoker = 'yes'
GROUP BY    region

%%sql
SELECT region, MIN(count) min_smoker
FROM    (
    SELECT      region, COUNT(*) count
    FROM        insurance
    WHERE       smoker = 'yes'
    GROUP BY    region
)]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


8. What is the average charge from female?

In [24]:

%%sql
SELECT  sex, AVG(charges) avg_charge
FROM    insurance
WHERE   sex = 'female'

 * sqlite:///insurance.db
Done.


sex,avg_charge
female,12569.57884383534


### Summary



1. 115 Female are smokers while 159 Male are smokers
2. Southeast has the highest number of non smokers, 273. Northwest and Southwest has 58 smokers, the lowest across all region.
3. Northwest and Southwest are the region with average charge lower than the overall average charge.
4. Southeast has the highest number of smokers
5. The average charges from Female is $12,570